### Reranking Hybrid Search Statergies

Re-ranking is a second-stage filtering process in retrieval systems, especially in RAG pipelines, where we:

1. First use a fast retriever (like BM25, FAISS, hybrid) to fetch top-k documents quickly.

2. Then use a more accurate but slower model (like a cross-encoder or LLM) to re-score and reorder those documents by relevance to the query.

👉 It ensures that the most relevant documents appear at the top, improving the final answer from the LLM.

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser 

In [ ]:
## load text file
loader=TextLoader("langchain_sample.txt")
raw_docs=loader.load()

# Split text into document chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)
docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(metadata={'source': 'langchain_sample.txt'}, page_content='Retrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional 

In [3]:
## user query
query="How can i use langchain to build an application with memory and tools?"

In [19]:
### FAISS and Huggingface model Embeddings

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(docs,embedding_model)
retriever=vectorstore.as_retriever(search_kwargs={"k":8})

In [ ]:
## OpenAI Embedding
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)
retriever_openai=vectorstore_openai.as_retriever(search_kwargs={"k":8})

In [21]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BBFD92AC40>, search_kwargs={'k': 8})

In [22]:
retriever_openai

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001BBFD9729F0>, search_kwargs={'k': 8})

In [23]:
## prompt and use the llm
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm=init_chat_model("groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BBFDAD0190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BBFDAD0690>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [24]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

User Question: "{question}"

Documents:
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [25]:
retrieved_docs=retriever.invoke(query)
retrieved_docs

[Document(id='709da153-eaee-411b-bd17-b5fc443be7da', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='a217edf2-c10c-40ff-9317-37c4e3388096', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='ec8fd256-9859-4aa8-becb-b505799be510', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [26]:
chain=prompt| llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\nUser Question: "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user\'s question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BBFDAD0190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BBFDAD0690>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [27]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)

In [28]:
doc_lines

['1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and comp

In [29]:
formatted_docs

'1. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n2. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n3. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n4. FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed ind

In [30]:
response=chain.invoke({"question":query,"documents":formatted_docs})
response

"2,1,4,5,6  \n\n\n**Explanation:**\n\n* **Document 2** directly addresses the user's question by outlining LangChain's capabilities for building applications with LLMs, including memory and tools. \n* **Document 1**  explains memory and tool integration within LangChain, which are key components of the user's query.\n* **Document 4**  focuses on agents, a specific type of tool-using LangChain application, further relevant to the question.\n* **Document 5** discusses Retrieval-Augmented Generation (RAG), another concept related to using tools and external knowledge within LangChain applications.\n* **Document 6** touches on hybrid retrieval, which could be relevant to building applications that leverage tools for information retrieval.\n\n\nThe other documents are less relevant as they focus on specific aspects of LangChain or related technologies without directly addressing the core question of building applications with memory and tools.\n"

In [31]:
# Step 5: Parse and rerank
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[1, 0, 3, 4]

In [32]:
retrieved_docs

[Document(id='709da153-eaee-411b-bd17-b5fc443be7da', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='a217edf2-c10c-40ff-9317-37c4e3388096', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='ec8fd256-9859-4aa8-becb-b505799be510', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging F

In [33]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]
reranked_docs

[Document(id='a217edf2-c10c-40ff-9317-37c4e3388096', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='709da153-eaee-411b-bd17-b5fc443be7da', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='d075ce19-78fc-4392-bde0-0a6a8a45a7ec', metadata={'source': 'langchain_sample.txt'}, page_content='FAISS is a popular library used for fast approximate nearest neighbor search 

In [34]:
# Step 6: Show results
print("\n📊 Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}")


📊 Final Reranked Results:

Rank 1:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 3:
FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering w

## **Notes - Reranking in Information Retrieval**

> Reranking is an advanced retrieval technique used to **improve the relevance of search results** by applying a secondary ranking model to an initial set of candidate documents retrieved from a primary search system (such as TF-IDF, BM25, or a dense vector retriever like FAISS or Pinecone).
It plays a crucial role in **Retrieval-Augmented Generation (RAG)** and **hybrid retrieval pipelines**, where initial recall is broad, and reranking helps refine precision.


**1. What is Reranking?**

Reranking is a **two-stage retrieval process**:

1. **Initial Retrieval (Stage 1):**
   A fast retriever (like BM25 or a dense vector retriever) fetches the *top N* most relevant documents based on similarity scores.

2. **Reranking (Stage 2):**
   A more sophisticated model (often transformer-based, such as **cross-encoders**) re-evaluates those N documents against the query to produce a **more accurate relevance score** and reorder them.

This approach balances **efficiency and accuracy** — the first stage is fast and broad, while the second stage is slow but precise.

---

**2. Why Use Reranking?**

* **Improves Precision:**
  Filters out false positives from the first-stage retriever.

* **Enhances Contextual Understanding:**
  Neural rerankers (e.g., BERT-based models) understand semantic meaning beyond keyword overlap.

* **Balances Performance:**
  Running deep models on all documents is costly; reranking applies them only to a small candidate set.

* **Improves RAG Outputs:**
  For systems like LangChain RAG pipelines, reranking ensures only the most relevant context is passed to the LLM.

---

**3. Common Reranking Approaches**

**a. Cross-Encoder Reranking**

A **cross-encoder** takes both the query and document as joint input (concatenated) and computes a **single similarity score** through a transformer model.

* **Example Models:**

  * `cross-encoder/ms-marco-MiniLM-L-6-v2`
  * `cross-encoder/ms-marco-TinyBERT-L-2-v2`
  * `nli-deberta-v3-base`

* **Advantages:**

  * Highest accuracy
  * Deep contextual understanding

* **Disadvantages:**

  * Computationally expensive
  * Not scalable for large datasets

---

**b. Bi-Encoder + Cross-Encoder (Hybrid Reranking)**

A **bi-encoder** first retrieves documents using vector embeddings (fast).
Then, a **cross-encoder** reranks the top results.

This hybrid approach is standard in modern systems like **ColBERT**, **Haystack**, or **LangChain retrievers**.

* **Stage 1:** Dense retrieval using embeddings (e.g., BERT, OpenAI Embeddings)
* **Stage 2:** Reranking using cross-encoder for top *k* results

---

**c. LLM-Based Reranking**

Large Language Models (LLMs) such as **GPT-4**, **Claude**, or **Llama 3** can be used to rerank retrieved documents by **evaluating their semantic match** with the query.

* **Prompt Example:**

  ```
  Given the query: "Explain quantum entanglement"
  And the following documents, rank them in order of relevance.
  ```

* **Use Case:**
  When explainability and reasoning-based ranking are required.

* **Trade-off:**
  Very high computational cost but superior contextual precision.

---

**4. Reranking Metrics**

When evaluating rerankers, common **information retrieval metrics** include:

| Metric                                           | Description                                               |
| ------------------------------------------------ | --------------------------------------------------------- |
| **MRR (Mean Reciprocal Rank)**                   | Measures how high the first relevant document ranks.      |
| **nDCG (Normalized Discounted Cumulative Gain)** | Evaluates ranking quality considering position relevance. |
| **Precision@K**                                  | Fraction of top K retrieved items that are relevant.      |
| **Recall@K**                                     | Fraction of relevant items retrieved among top K.         |

---

**5. Example Workflow in LangChain**

```python
from langchain.retrievers import BM25Retriever, CrossEncoderReranker
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Step 1: Initialize BM25 retriever
bm25_retriever = BM25Retriever.from_documents(documents)

# Step 2: Initialize cross-encoder reranker
reranker = CrossEncoderReranker(model_name="cross-encoder/ms-marco-MiniLM-L-6-v2")

# Step 3: Wrap retriever with reranker
hybrid_retriever = bm25_retriever | reranker  # LCEL pipeline-style

# Step 4: Use in a RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini"),
    retriever=hybrid_retriever,
)
```

---

**6. Practical Applications**

* **Search Engines:**
  Rerank top 100 BM25 results using cross-encoder to improve click relevance.

* **Question Answering (RAG):**
  Improve retrieved document quality before passing to the LLM.

* **E-commerce:**
  Rerank product results based on semantic and behavioral similarity.

* **Legal and Biomedical Search:**
  Ensure retrieved documents are contextually aligned and domain-accurate.

---

**7. Best Practices**

* Use **dense retrievers for recall**, **cross-encoders for precision**.
* Limit reranking to **top 50–200 candidates** to avoid latency spikes.
* Precompute embeddings and cache intermediate results.
* Evaluate performance regularly using **nDCG** and **Recall@K**.
* Experiment with **hybrid rerankers** (dense + sparse + cross-encoder) for optimal balance.

---

**Summary Table**

| Stage | Component | Model Type              | Purpose                                     |
| ----- | --------- | ----------------------- | ------------------------------------------- |
| 1️⃣   | Retriever | BM25 / Dense Embeddings | Fast candidate retrieval                    |
| 2️⃣   | Reranker  | Cross-Encoder / LLM     | Accurate semantic ranking                   |
| 3️⃣   | Generator | LLM / RAG               | Produces final answer using top-ranked docs |


